**CUDA**

Евгений Борисов <esborisov@sevsu.ru>

---

In [1]:
import pycuda

# help(pycuda)
pycuda.VERSION

(2022, 1)

In [2]:
import pycuda.autoinit
import pycuda.driver as cuda

(free,total) = cuda.mem_get_info()

print('total %iMB'%(total//1024**2))
print('free %iMB - %.02f%% '%(free//1024**2,free*100/total))

total 6075MB
free 5563MB - 91.57% 


In [3]:
cuda.Device.count()

1

In [4]:
device = cuda.Device(0)

print('device:',device.name())

device: NVIDIA GeForce GTX 1060 6GB


In [5]:
from tabulate import tabulate 

attrs = device.get_attributes()
tabulate( [ [str(k),int(v)] for k,v in attrs.items() ],tablefmt='html', )

ASYNC_ENGINE_COUNT,2
CAN_MAP_HOST_MEMORY,1
CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM,1
CLOCK_RATE,1847500
COMPUTE_CAPABILITY_MAJOR,6
COMPUTE_CAPABILITY_MINOR,1
COMPUTE_MODE,0
COMPUTE_PREEMPTION_SUPPORTED,1
CONCURRENT_KERNELS,1
CONCURRENT_MANAGED_ACCESS,1
DIRECT_MANAGED_MEM_ACCESS_FROM_HOST,0


In [6]:
print(
    '%s\n  CUDA COMPUTE CAPABILITY: %i.%i'%(
    device.name(),
    attrs[pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MAJOR],
    attrs[pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MINOR]
    )
)

NVIDIA GeForce GTX 1060 6GB
  CUDA COMPUTE CAPABILITY: 6.1


----

In [7]:
import numpy as np
from pycuda.compiler import SourceModule

In [8]:
mod = SourceModule('''
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
''')

multiply_them = mod.get_function('multiply_them')

In [9]:
n = 1024 # MAX_THREADS_PER_BLOCK: 1024

a = np.random.randn(n).astype(np.float32)
b = np.random.randn(n).astype(np.float32)
z = np.zeros(n, dtype=np.float32)

multiply_them( cuda.Out(z), cuda.In(a), cuda.In(b), block=(n,1,1), grid=(1,1))

In [10]:
assert (z == a*b).all()

In [11]:
z

array([-0.07560632,  0.10713323,  0.3355968 , ...,  0.48046944,
       -0.2569455 , -2.4029944 ], dtype=float32)